# Retrieval Augmented Generations

Penulis : Joko Eliyanto
Hak Cipta : Hobi Data

## Apa itu RAG?

Model Large Language Model yang sudah dilatih(pretrained) telah terbukti mampu menyimpan pengetahuan yang bisa kita tanyakan hanya dengan bahasa natural seperti kita bercakap-cakap dengan teman kita. Meskipun demikian, LLM tetap terbatas pada pengetahuan yang telah di training pada model tersebut. Fine tuning bukanlah pilihan yang murah, maka penambahan pengetahuan dengan fine tuning tidak memungkinkan untuk semua orang. Diperlukan sebuah metode yang dapat digunakan untuk memperkaya pengetahuan namun dengan biaya yang lebih terjangkau. 

Retrieval Augmented Generation, merupakan pengkayaan generasi jawaban oleh LLM dengan menggunakan hasil retrieval dari database vector. Sebuah informasi tambahan di luar pengetahuan LLM diproses menjadi sebuah vector embedding dan kemudian disimpan pada suatu vector database. Kemudian query untuk mencari informasi akan melalui proses pencarian di dalam vector database(retrieval) yang kemudian hasilnya akan dijadikan sebagai context untuk generasi jawaban akhir. 

### Arsitektur RAG

...

## Load and Document

In [16]:
from langchain_community.document_loaders import PyPDFLoader
import time

file_path = "data/modul_streamlit.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

In [18]:
docs[:2]

[Document(metadata={'producer': 'Skia/PDF m142', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', 'creationdate': '2025-11-28T01:08:38+00:00', 'title': 'Membuat Web App Data Science Menggunakan Streamlit', 'moddate': '2025-11-28T01:08:38+00:00', 'source': 'data/modul_streamlit.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}, page_content='Membuat Web App Data Science\nMenggunakan Streamlit\nA. Pengenalan Streamlit\n1. Apa itu Streamlit?\nStreamlit merupakan library Python open-source yang memungkinkan kita membuat aplikasi\nweb interaktif khusus untuk Data Science dan Machine Learning dalam waktu singkat.\nDengan streamlit, kita tidak perlu menguasai HTML, CSS, atau JavaScript untuk membuat\nwebsite. Cukup dengan bahasa pemrogram Python yang sudah kita ketahui, kita dapat\nmembuat aplikasi web yang interaktif dan menarik.\n2. Instalasi & Setup\nSebelum memulai, kita perlu melakukan instalasi strea

## Vector Database

### Model Embedding

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings

emb = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",   # atau BGE, Qwen, dll
    model_kwargs={"device": "cpu"},           # "cuda" kalau ada GPU
    encode_kwargs={"normalize_embeddings": True}
)

In [8]:
# Test
v = emb.embed_query("hello world")
print(len(v))

768


### FAIS Vector Database Memory

In [29]:
from langchain_community.vectorstores import FAISS

# help(FAISS)

In [23]:
vector_store = FAISS.from_documents(documents = docs, embedding = emb)

# help(vector_store)

In [25]:
vector_store.similarity_search(query="apa itu streamlit?",k=3)

[Document(id='ffe3da21-3a0e-41d3-bf3f-a273b3d0b15d', metadata={'producer': 'Skia/PDF m142', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', 'creationdate': '2025-11-28T01:08:38+00:00', 'title': 'Membuat Web App Data Science Menggunakan Streamlit', 'moddate': '2025-11-28T01:08:38+00:00', 'source': 'data/modul_streamlit.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}, page_content='Membuat Web App Data Science\nMenggunakan Streamlit\nA. Pengenalan Streamlit\n1. Apa itu Streamlit?\nStreamlit merupakan library Python open-source yang memungkinkan kita membuat aplikasi\nweb interaktif khusus untuk Data Science dan Machine Learning dalam waktu singkat.\nDengan streamlit, kita tidak perlu menguasai HTML, CSS, atau JavaScript untuk membuat\nwebsite. Cukup dengan bahasa pemrogram Python yang sudah kita ketahui, kita dapat\nmembuat aplikasi web yang interaktif dan menarik.\n2. Instalasi & Setup\nSebelum me

In [26]:
vector_store.similarity_search_with_relevance_scores(query="apa itu streamlit?",k=3)

[(Document(id='ffe3da21-3a0e-41d3-bf3f-a273b3d0b15d', metadata={'producer': 'Skia/PDF m142', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', 'creationdate': '2025-11-28T01:08:38+00:00', 'title': 'Membuat Web App Data Science Menggunakan Streamlit', 'moddate': '2025-11-28T01:08:38+00:00', 'source': 'data/modul_streamlit.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}, page_content='Membuat Web App Data Science\nMenggunakan Streamlit\nA. Pengenalan Streamlit\n1. Apa itu Streamlit?\nStreamlit merupakan library Python open-source yang memungkinkan kita membuat aplikasi\nweb interaktif khusus untuk Data Science dan Machine Learning dalam waktu singkat.\nDengan streamlit, kita tidak perlu menguasai HTML, CSS, atau JavaScript untuk membuat\nwebsite. Cukup dengan bahasa pemrogram Python yang sudah kita ketahui, kita dapat\nmembuat aplikasi web yang interaktif dan menarik.\n2. Instalasi & Setup\nSebelum m

## Retriever

In [31]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# help(retriever)

In [12]:
retriever.invoke("apa itu streamlit?")

[Document(id='9c2d6cb6-849d-45e7-82ff-577ac8a3621e', metadata={}, page_content='ibrary streamlit dan memberikan alias st\nuntuk memudahkan penulisan kode.\nst.title("Aplikasi Pertamaku"): Menambahkan judul pada aplikasi web.\nst.write("Hello World! Selamat datang di tutorial Streamlit."):\nMenambahkan teks pada aplikasi web.\nLangkah 3: Menjalankan Aplikasi Buka terminal, lalu ketikan perintah berikut (pastikan\nAnda berada di folder yang sama dengan app.py):\nPerintah ini akan menjalankan aplikasi streamlit. Browser akan terbuka dan menampilkan\nhalaman web baru. Berikut adalah tampilan aplikasi Hello World versi Streamlit:\nimport streamlit as st\nst.title("Aplikasi Pertamaku")\nst.write("Hello World! Selamat datang di tutorial Streamlit.")\nstreamlit run app.py\nB. Menampilkan Informasi Dasar\nSetelah berhasil menjalankan aplikasi Hello World, langkah selanjutnya adalah memahami\nbagaimana menyusun konten. Dalam pembuatan aplikasi web atau dashboard untuk\nmenampilkan data, struktur

## Large Language Model

In [33]:
from langchain_community.llms import OpenAI

# LLM
llm = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model_name="qwen/qwen3-vl-4b"   # isi model LM Studio
)

# help(llm)

In [35]:
response = llm.invoke("kamu tau streamlit?")

response

' \n\nTentu! 😊  \n**Streamlit** adalah sebuah **framework Python** yang dirancang untuk membuat **aplikasi web interaktif** dengan **cara yang cepat, sederhana, dan intuitif** — terutama untuk **analisis data, visualisasi, dan machine learning**.\n\n---\n\n### 🌟 Fitur Utama Streamlit:\n\n1. **Cepat dibuat & dijalankan**  \n   - Dengan hanya menulis beberapa baris kode, kamu bisa membuat aplikasi web yang bisa diakses langsung dari terminal (`streamlit run app.py`).\n\n2. **Komponen interaktif**  \n   - Bisa menambahkan tombol, input teks, selectbox, slider, map, grafik, dll.\n\n3. **Visualisasi data**  \n   - Mudah mengintegrasikan **matplotlib, seaborn, plotly, altair**, dan lainnya.\n\n4. **Dukungan ML**  \n   - Cocok untuk menampilkan model ML, prediksi, dan dashboard model.\n\n5. **Deploying mudah**  \n   - Bisa di-deploy ke **Streamlit Cloud**, **Heroku**, **Verc'

## RAG Chain

In [40]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

In [41]:
# Custom QA prompt
prompt = PromptTemplate.from_template("""
Jawab dengan jawaban profesional.
Gunakan konteks berikut untuk menjawab pertanyaan:

Konteks:
{context}

Pertanyaan:
{question}

Jawaban:
""")

# LCEL pipeline
rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

In [43]:
response = rag_chain.invoke("apa itu streamlit?")

In [44]:
response

'Streamlit adalah sebuah library Python open-source yang dirancang untuk membantu pengguna membuat aplikasi web interaktif khusus untuk Data Science dan Machine Learning secara cepat dan mudah. Dengan Streamlit, pengguna tidak perlu memiliki pengetahuan mendalam tentang HTML, CSS, atau JavaScript — cukup menulis kode dalam bahasa Python yang sudah dikenal, pengguna dapat mengembangkan aplikasi web yang interaktif dan menarik. Streamlit juga memungkinkan pengguna untuk menampilkan data dan membuat dashboard yang fungsional tanpa harus membangun struktur halaman web dari awal.\n\nStreamlit memiliki versi 1.50.0, yang dapat diperiksa dengan perintah `streamlit --version` setelah instalasi berhasil. Selain itu, Streamlit dapat dijalankan dengan perintah `streamlit run app.py`, yang akan membuka browser dan menampilkan aplikasi web yang telah dibuat.\n\nDengan demikian, Streamlit menjadi alat yang sangat membantu bagi para praktisi Data Science dan Machine Learning yang ingin mengembangkan 

## RAG Function

In [56]:
# help(llm)

In [48]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""
Jawab dengan jawaban profesional.
Gunakan konteks berikut untuk menjawab pertanyaan:

Konteks:
{context}

Pertanyaan:
{question}

Jawaban:
""")


In [49]:
questions = "Apa itu streamlit?"

In [50]:
retrieval_result = vector_store.similarity_search_with_score(query = questions, k=3)
retrieval_result

[(Document(id='ffe3da21-3a0e-41d3-bf3f-a273b3d0b15d', metadata={'producer': 'Skia/PDF m142', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', 'creationdate': '2025-11-28T01:08:38+00:00', 'title': 'Membuat Web App Data Science Menggunakan Streamlit', 'moddate': '2025-11-28T01:08:38+00:00', 'source': 'data/modul_streamlit.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}, page_content='Membuat Web App Data Science\nMenggunakan Streamlit\nA. Pengenalan Streamlit\n1. Apa itu Streamlit?\nStreamlit merupakan library Python open-source yang memungkinkan kita membuat aplikasi\nweb interaktif khusus untuk Data Science dan Machine Learning dalam waktu singkat.\nDengan streamlit, kita tidak perlu menguasai HTML, CSS, atau JavaScript untuk membuat\nwebsite. Cukup dengan bahasa pemrogram Python yang sudah kita ketahui, kita dapat\nmembuat aplikasi web yang interaktif dan menarik.\n2. Instalasi & Setup\nSebelum m

In [52]:
context = [element[0].page_content for element in retrieval_result]
context

['Membuat Web App Data Science\nMenggunakan Streamlit\nA. Pengenalan Streamlit\n1. Apa itu Streamlit?\nStreamlit merupakan library Python open-source yang memungkinkan kita membuat aplikasi\nweb interaktif khusus untuk Data Science dan Machine Learning dalam waktu singkat.\nDengan streamlit, kita tidak perlu menguasai HTML, CSS, atau JavaScript untuk membuat\nwebsite. Cukup dengan bahasa pemrogram Python yang sudah kita ketahui, kita dapat\nmembuat aplikasi web yang interaktif dan menarik.\n2. Instalasi & Setup\nSebelum memulai, kita perlu melakukan instalasi streamlit. Ketik perintah berikut pada\nterminal dan tekan Enter:\nUntuk mengecek apakah instalasi berhasil, ketik perintah berikut pada terminal dan tekan\nEnter:\nOutputnya dari perintah ini akan menampilkan nomor versi streamlit yang sudah terinstall\ndi komputer Anda. Berikut adalah contoh outputnya:\n3. Membuat Aplikasi Hello World Menggunakan Streamlit\nPada bagian ini, kita akan membuat aplikasi hello world menggunakan stre

In [54]:
source = [element[0].metadata['source'] + ", page: " + element[0].metadata['page_label'] for element in retrieval_result]
source

['data/modul_streamlit.pdf, page: 1',
 'data/modul_streamlit.pdf, page: 11',
 'data/modul_streamlit.pdf, page: 2']

In [57]:
prompt = template.format(context = context, question = questions)

prompt

'\nJawab dengan jawaban profesional.\nGunakan konteks berikut untuk menjawab pertanyaan:\n\nKonteks:\n[\'Membuat Web App Data Science\\nMenggunakan Streamlit\\nA. Pengenalan Streamlit\\n1. Apa itu Streamlit?\\nStreamlit merupakan library Python open-source yang memungkinkan kita membuat aplikasi\\nweb interaktif khusus untuk Data Science dan Machine Learning dalam waktu singkat.\\nDengan streamlit, kita tidak perlu menguasai HTML, CSS, atau JavaScript untuk membuat\\nwebsite. Cukup dengan bahasa pemrogram Python yang sudah kita ketahui, kita dapat\\nmembuat aplikasi web yang interaktif dan menarik.\\n2. Instalasi & Setup\\nSebelum memulai, kita perlu melakukan instalasi streamlit. Ketik perintah berikut pada\\nterminal dan tekan Enter:\\nUntuk mengecek apakah instalasi berhasil, ketik perintah berikut pada terminal dan tekan\\nEnter:\\nOutputnya dari perintah ini akan menampilkan nomor versi streamlit yang sudah terinstall\\ndi komputer Anda. Berikut adalah contoh outputnya:\\n3. Membu

In [60]:
result = llm.invoke(prompt)

In [61]:
result

'**Streamlit** adalah library Python open-source yang dirancang khusus untuk membuat aplikasi web interaktif untuk keperluan Data Science dan Machine Learning. Dengan Streamlit, pengguna dapat mengembangkan aplikasi web secara cepat tanpa perlu memahami HTML, CSS, atau JavaScript — cukup dengan menulis kode dalam bahasa Python yang sudah dikenal. Streamlit memungkinkan pengguna untuk membangun antarmuka pengguna yang dinamis dan interaktif, serta memungkinkan interaksi langsung dengan data dan model yang dibangun, seperti menekan tombol, memilih parameter, atau memasukkan data. Streamlit juga menyediakan berbagai widget interaktif yang memungkinkan pengguna untuk menjalankan ulang skrip secara otomatis saat terjadi perubahan, sehingga memudahkan pengembangan dan pengujian aplikasi secara iteratif. Versi terbaru Streamlit yang tersedia saat ini adalah **Streamlit, version 1.50.0**. \n\nDengan demikian, Streamlit menjadi alat yang sangat ideal bagi para praktisi Data Science dan Machine 

In [73]:
def rag(query, k=3):
    retrieval_result = vector_store.similarity_search_with_score(query = query, k=k)

    context = [element[0].page_content for element in retrieval_result]
    source = [element[0].metadata['source'] + ", page: " + element[0].metadata['page_label'] for element in retrieval_result]

    prompt = template.format(context = context, question = query)

    response_text = llm.invoke(prompt)

    return retrieval_result, response_text, source

In [74]:
retrieval_result, response_text, source = rag(query="Bagaimana membuat layout di streamlit?")

In [75]:
retrieval_result

[(Document(id='ffe3da21-3a0e-41d3-bf3f-a273b3d0b15d', metadata={'producer': 'Skia/PDF m142', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', 'creationdate': '2025-11-28T01:08:38+00:00', 'title': 'Membuat Web App Data Science Menggunakan Streamlit', 'moddate': '2025-11-28T01:08:38+00:00', 'source': 'data/modul_streamlit.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}, page_content='Membuat Web App Data Science\nMenggunakan Streamlit\nA. Pengenalan Streamlit\n1. Apa itu Streamlit?\nStreamlit merupakan library Python open-source yang memungkinkan kita membuat aplikasi\nweb interaktif khusus untuk Data Science dan Machine Learning dalam waktu singkat.\nDengan streamlit, kita tidak perlu menguasai HTML, CSS, atau JavaScript untuk membuat\nwebsite. Cukup dengan bahasa pemrogram Python yang sudah kita ketahui, kita dapat\nmembuat aplikasi web yang interaktif dan menarik.\n2. Instalasi & Setup\nSebelum m

In [76]:
response_text

'Dalam Streamlit, layout atau tata letak aplikasi dibuat menggunakan dua fitur utama: **Sidebar** dan **Columns**. Fitur-fitur ini memungkinkan pengguna untuk menata elemen-elemen aplikasi secara profesional dan user-friendly, agar tampilan aplikasi tidak hanya fungsional tetapi juga estetis dan rapi.\n\n### 1. **Sidebar**\nSidebar digunakan untuk menambahkan elemen navigasi atau kontrol yang dapat diakses di sisi kiri aplikasi. Misalnya, pengguna dapat memilih opsi, mengatur parameter, atau mengakses menu lain tanpa mengganggu tampilan utama.\n\nContoh penggunaan:\n```python\nst.sidebar.title("Menu Navigasi")\nst.sidebar.radio("Pilih Opsi", ["Ops 1", "Ops 2", "Ops 3"])\n```\n\n### 2. **Columns**\nColumns digunakan untuk membagi tampilan aplikasi menjadi beberapa kolom atau bagian secara horizontal. Dengan columns, pengguna dapat menampilkan lebih dari satu elemen secara bersamaan dalam satu baris, misalnya grafik dan teks, atau dua tombol kontrol.\n\n'

In [77]:
source

['data/modul_streamlit.pdf, page: 1',
 'data/modul_streamlit.pdf, page: 17',
 'data/modul_streamlit.pdf, page: 18']